In [ ]:
import os
import sys

sys.path.append(os.path.abspath("../src"))

In [ ]:
import torch


def bool_matmul(a, b):
    a = a.to(torch.uint8)
    b = b.to(torch.uint8)
    c = a @ b
    return c.to(torch.bool)

Sign of derivative can be derived from the weight value, so boolean results are enough.

In [ ]:
def nor_and_eval(weights, inputs):
    weights = weights.to(torch.int32)
    inputs = inputs.to(torch.int32)
    or_and = (weights @ inputs).to(torch.bool)
    return ~or_and


def nor_and_grad(weights, inputs):
    weights = weights.to(torch.int32)
    inputs = inputs.transpose(-1, -2)
    prods = weights * inputs.to(torch.int32)
    residues = prods.sum(dim=1, keepdim=True) - prods
    return (~residues.to(torch.bool) & inputs).to(torch.bool)

In [ ]:
_weights = torch.tensor([[False, True], [True, False]], dtype=torch.bool)
inputs = torch.tensor([[True], [True]], dtype=torch.bool)
nor_and_eval(_weights, inputs)
nor_and_grad(_weights, inputs)

tensor([[False,  True],
        [ True, False]])

In [ ]:
class NorAndLinear:
    def __init__(self, weights):
        self._weights = weights

    def eval(self, inputs):
        weights = self._weights.to(torch.int32)
        inputs = inputs.to(torch.int32)
        or_and = (weights @ inputs).to(torch.bool)
        return ~or_and

    def grad(self, inputs):
        weights = self._weights.to(torch.int32)
        inputs = inputs.transpose(-1, -2)
        prods = weights * inputs.to(torch.int32)
        residues = prods.sum(dim=1, keepdim=True) - prods
        return (~residues.to(torch.bool) & inputs).to(torch.bool)

In [2]:
import torch

# Example tensor
x = torch.tensor([[0, 0, 3], [0, 0, 0], [4, 0, 0]], dtype=torch.uint32)

# Compute the row-wise check
result = (x.any(dim=1, keepdim=True)).to(torch.uint32).T  # (n, 1) -> (1, n)

print(result)

tensor([[1, 0, 1]], dtype=torch.uint32)


In [ ]:
import torch

# Example tensor
x = torch.tensor([[0, 0, 0], [0, 0, 0], [1, 2, 3], [4, 0, 0]], dtype=torch.uint32)

# Compute the row-wise check
result = x.any(dim=1, keepdim=True).to(torch.uint32)  # Shape (n, 1)

print(result)

tensor([[0],
        [0],
        [1],
        [1]], dtype=torch.uint32)


In [ ]:
import bitwise

import importlib

importlib.reload(bitwise)

<module 'bitwise' from '/Users/simongolbert/Desktop/research/src/bitwise/__init__.py'>

In [10]:
import bitwise


bitwise.tensor([["110"]])._data

tensor([[[3221225472]]], dtype=torch.uint32)